In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
# Initialize lists to store phone models and prices
phone_models = []
phone_prices = []
phone_condition = []
phone_discount = []
phone_prevprice = []
phone_rating = []

# Specify the number of pages you want to scrape
num_pages = 10  # Change this to the number of pages you want to scrape

for page in range(1,20):
    # Define the eBay URL for the current page
    url = f"https://www.ebay.com/sch/i.html?_from=R40&_trksid=p4432023.m570.l1313&_nkw=phones&_sacat=0&_ipg=240&_pgn={page}"

    # Send an HTTP GET request to the eBay URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, "html.parser")

        # Find phone listings
        phone_listings = soup.find_all("div", class_="s-item__info")

        # Extract data from each listing on the current page
        for listing in phone_listings:
            try:
                model = listing.find("div", class_="s-item__title").text.strip()
                price = listing.find("span", class_="s-item__price").text.strip()
                condition = listing.find("span", class_="SECONDARY_INFO").text.strip()
                discount = listing.find("span", class_="s-item__discount" ).text.strip()
                prevprice = listing.find("span", class_="s-item__trending-price").text.strip()
            
                star_rating_elem = listing.find_all("svg", class_="icon--star-filled-16")
                #star_rating = ""
                star_icons = listing.find_all("svg", class_="icon--star-half-16-colored")
                
                if star_icons:
                    star_rating = len(star_rating_elem) + 0.5
                    
                elif star_rating_elem:
                    star_rating = len(star_rating_elem)
                    
                else:
                    star_rating = "N/A"
            
            
            
                phone_models.append(model)
                phone_prices.append(price)
                phone_condition.append(condition)
                phone_discount.append(discount)
                phone_prevprice.append(prevprice)
                phone_rating.append(star_rating)
            except AttributeError:
                # Handle cases where data is not found in a listing
                pass
    else:
        print(f"Failed to retrieve page {page}")

In [4]:
Offer = pd.DataFrame(phone_discount)

In [5]:
Previous_Price = pd.DataFrame(phone_prevprice)

In [6]:
Model = pd.DataFrame(phone_models)

In [7]:
Price = pd.DataFrame(phone_prices)

In [8]:
Condition = pd.DataFrame(phone_condition)

In [9]:
Rating = pd.DataFrame(phone_rating)

In [10]:
data = pd.concat([Price, Model, Condition, Offer, Previous_Price, Rating], axis=1)

In [11]:
data.columns = ['Price', 'Model', 'Condition', 'Offer', 'Previous Price', 'Rating']

In [12]:
data['Previous Price'] = data['Previous Price'].str.replace('[^0-9.]', '', regex=True)

In [13]:
data

,Price,Model,Condition,Offer,Previous Price,Rating
0,$389.95,Apple iPhone 13 mini 128GB Factory Unlocked AT...,Good - Refurbished,44% off,699.00,5
1,$109.47,"S23 Ultra Smartphone 7.3"" 4+64GB Android Facto...",Brand New,8% off,118.99,N/A
2,$137.07,"S23 Ultra 8+256GB Smartphone 7.3"" Factory Unlo...",Brand New,8% off,148.99,N/A
3,$189.99,Samsung Galaxy S21 5G 128GB G991U Fully Unloc...,Pre-Owned,76% off,799.99,N/A
4,$197.99,Samsung Galaxy S21 5G 128GB G991U Fully Unloc...,Good - Refurbished,75% off,799.99,4.5
...,...,...,...,...,...,...
844,$574.95,Apple iPhone 12 Pro Max 256GB Verizon Smartpho...,Very Good - Refurbished,52% off,1199.00,5
845,$19.99,Samsung GT-E2121B Blue Cell Phone Internationa...,Brand New,50% off,39.99,N/A
846,$434.79,"Apple iPhone 12 Pro, A2341, 256GB, Gold, Unloc...",Good - Refurbished,60% off,1099.00,N/A
847,$78.84,Samsung Galaxy S7 G930 32GB Unlocked GSM AT&T ...,Open Box,5% off,82.99,4


In [14]:
S = None

def filter_data(change):
    global S
    selected_condition = change['new']
    S = data[data['Condition'] == selected_condition]

condition_dropdown = widgets.Dropdown(
    options=data['Condition'].unique(),
    description='Select Condition:'
)
condition_dropdown.observe(filter_data, names='value')

display(condition_dropdown)

Dropdown(description='Select Condition:', options=('Good - Refurbished', 'Brand New', 'Pre-Owned', 'Very Good …

In [29]:
S

,Price,Model,Condition,Offer,Previous Price,Rating
1,$109.47,"S23 Ultra Smartphone 7.3"" 4+64GB Android Facto...",Brand New,8% off,118.99,N/A
2,$137.07,"S23 Ultra 8+256GB Smartphone 7.3"" Factory Unlo...",Brand New,8% off,148.99,N/A
10,$59.99,2023 Factory Unlocked Android Cheap Cell Phone...,Brand New,25% off,79.99,N/A
12,$137.99,"Unlocked 7.3"" i14 Pro Max Android 13 Smartphon...",Brand New,8% off,149.99,N/A
21,$128.79,"2023 Reno10 Pro+ Smartphone 7.3"" 16GB+1TB Andr...",Brand New,8% off,139.99,N/A
...,...,...,...,...,...,...
841,$12.50,Samsung SGH-R200 (International) Olympics Mobi...,Brand New,50% off,24.99,N/A
842,$13.99,For Samsung Galaxy Z Flip 3 5G Heavy Duty Shoc...,Brand New,5% off,14.73,N/A
843,$22.49,Samsung SGH-N288 SLIM (AnyCall International) ...,Brand New,50% off,44.99,N/A
845,$19.99,Samsung GT-E2121B Blue Cell Phone Internationa...,Brand New,50% off,39.99,N/A


In [30]:
len(S)

210

In [31]:
dup = S[S.duplicated(subset=['Model'], keep=False)]

In [32]:
dup

,Price,Model,Condition,Offer,Previous Price,Rating
322,$132.99,New in Sealed Box Motorola Z4 XT1980-4 VERIZON...,Brand New,12% off,151.05,4.5
672,$155.99,New in Sealed Box Motorola Z4 XT1980-4 VERIZON...,Brand New,15% off,183.99,N/A
678,$54.00,NEW Doro 824C 8GB 4G LTE BELL MOBILITY UNLOCKE...,Brand New,94% off,838.99,4.5
723,$54.00,NEW Doro 824C 8GB 4G LTE BELL MOBILITY UNLOCKE...,Brand New,94% off,838.99,4.5
741,$262.99,Apple iPhone X 256G A1901 ATT T-MOB UNLOCKED S...,Brand New,15% off,308.99,4.5
800,$262.99,Apple iPhone X 256G A1901 ATT T-MOB UNLOCKED S...,Brand New,15% off,308.99,4.5


In [33]:
filtered_data = dup[~dup.duplicated(subset=['Model', 'Offer'], keep=False)]

In [34]:
unique_duplicates = [''] + list(filtered_data['Model'].unique())

In [35]:
unique_duplicates

['',
 'New in Sealed Box Motorola Z4 XT1980-4 VERIZON ONLY 128G GRAY Smartphone WF']

In [36]:
# Function to display data for a selected duplicate value
def display_duplicate_data(change):
    selected_value = change['new']
    duplicate_data = S[S['Model'] == selected_value]
    with output:
        output.clear_output()
        display(duplicate_data)

# Create a dropdown widget to select a duplicate value
dropdown = widgets.Dropdown(
    options=unique_duplicates,  # Include an empty option
    description='Select a duplicate value:'
)


# Create an output widget to display the selected duplicate data
output = widgets.Output()

# Link the SelectMultiple widget to the custom function
dropdown.observe(display_duplicate_data, names='value')

# Display the widgets
display(dropdown)
display(output)

Dropdown(description='Select a duplicate value:', options=('', 'New in Sealed Box Motorola Z4 XT1980-4 VERIZON…

Output()